# Inspect Data in CSV Files

In this Jupiter Notebook, we will try to understand the different columns in the 8 CSV files, located in the `covid-data` folder.

In [2]:
import pandas as pd
import csv
from collections import defaultdict
import uuid

### Read All CSV Files

In [3]:
countries_aggregated = pd.read_csv("covid-data/countries-aggregated.csv", parse_dates=['Date'])
key_countries_pivoted = pd.read_csv("covid-data/key-countries-pivoted.csv", parse_dates=['Date'])
reference = pd.read_csv("covid-data/reference.csv")
time_series_19_covid_combined = pd.read_csv("covid-data/time-series-19-covid-combined.csv", parse_dates=['Date'])
us_confirmed = pd.read_csv("covid-data/us-confirmed.csv", parse_dates=['Date'])
us_deaths = pd.read_csv("covid-data/us-deaths.csv", parse_dates=['Date'])
us_simplified = pd.read_csv("covid-data/us-simplified.csv", parse_dates=['Date'])
worldwide_aggregate = pd.read_csv("covid-data/worldwide-aggregate.csv", parse_dates=['Date'])

### Initial Main File: Countries Aggregated 

In [4]:
# Countries Aggregated
countries_aggregated.info()
countries_aggregated.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161568 entries, 0 to 161567
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date       161568 non-null  datetime64[ns]
 1   Country    161568 non-null  object        
 2   Confirmed  161568 non-null  int64         
 3   Recovered  161568 non-null  int64         
 4   Deaths     161568 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 6.2+ MB


,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-23,Afghanistan,0,0,0


### Creating New CSV Files with Appropriate Data

In [5]:
# Generate unique uuid for countries_aggregated
input_file = "covid-data/countries-aggregated.csv"
output_file = "covid-data/countries-aggregated-with-uuid.csv"

with open(input_file, 'r') as csvfile, open(output_file, 'w', newline='') as outfile:
    reader = csv.DictReader(csvfile)
    fieldnames = ['id'] + reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in reader:
        row['id'] = str(uuid.uuid4())
        writer.writerow(row)

In [6]:
# Generate filtered version without the date column
output_file = "covid-data/countries-aggregated-filtered.csv"

with open(input_file, 'r') as csvfile, open(output_file, 'w', newline='') as outfile:
    reader = csv.DictReader(csvfile)
    fieldnames = [field for field in reader.fieldnames if field != 'Date']

    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in reader:
        # Remove the 'date' column from each row
        del row['Date']
        writer.writerow(row)

In [7]:
# Generate csv with the max total confirmed, recovered and deaths
output_file = "covid-data/countries-aggregated-sum.csv"

# Dictionary to store the max entry data
max_entry_data = defaultdict(lambda: {'Total_Confirmed': 0, 'Total_Recovered': 0, 'Total_Deaths': 0})

# Read the data from the input CSV file and get the max entry for each country
with open(input_file, 'r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        country = row['Country']
        max_entry_data[country]['Total_Confirmed'] = max(max_entry_data[country]['Total_Confirmed'], int(row['Confirmed']))
        max_entry_data[country]['Total_Recovered'] = max(max_entry_data[country]['Total_Recovered'], int(row['Recovered']))
        max_entry_data[country]['Total_Deaths'] = max(max_entry_data[country]['Total_Deaths'], int(row['Deaths']))

# Write the max entry data to the output CSV file
with open(output_file, 'w', newline='') as outfile:
    fieldnames = ['Country', 'Total_Confirmed', 'Total_Recovered', 'Total_Deaths']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for country, data in max_entry_data.items():
        writer.writerow({
            'Country': country,
            'Total_Confirmed': data['Total_Confirmed'],
            'Total_Recovered': data['Total_Recovered'],
            'Total_Deaths': data['Total_Deaths']
        })

In [8]:
# Simple test of max confirmed cases
df_sum = pd.read_csv("covid-data/countries-aggregated-sum.csv")
max_confirmed = df_sum['Total_Confirmed'].max()
country_with_max_confirmed = df_sum[df_sum['Total_Confirmed'] == max_confirmed]['Country'].values[0]
print("Max confirmed cases:", max_confirmed)
print("Country with max confirmed cases:", country_with_max_confirmed)

Max confirmed cases: 80625120
Country with max confirmed cases: US


In [9]:
# Generate filtered version with only the specified columns
input_ref_file = "covid-data/reference.csv"
output_file = "covid-data/reference-filtered.csv"
columns_to_keep = ['UID', 'iso3', 'Admin2', 'Province_State', 'Lat', 'Long_', 'Population']

with open(input_ref_file, 'r') as csvfile, open(output_file, 'w', newline='') as outfile:
    reader = csv.DictReader(csvfile)
    writer = csv.DictWriter(outfile, fieldnames=columns_to_keep)
    writer.writeheader()
    
    for row in reader:
        if not row['iso3']:
            row['iso3'] = "NUL"
        if not row['Admin2']:
            row['Admin2'] = "NUL"
        if not row['Population']:
            row['Population'] = 0
        
        filtered_row = {key: row[key] for key in columns_to_keep}
        writer.writerow(filtered_row)

In [10]:
# Generate a new column for the continent of all the countries
input_sum_file = "covid-data/countries-aggregated-sum.csv"
output_file = "covid-data/countries-aggregated-sum-continents.csv"

# Dictionary to map countries to continents
countries_to_continents = {
    "Afghanistan": "Asia",
    "Albania": "Europe",
    "Algeria": "Africa",
    "Andorra": "Europe",
    "Angola": "Africa",
    "Antarctica": "Antarctica",
    "Antigua and Barbuda": "North America",
    "Argentina": "South America",
    "Armenia": "Asia",
    "Australia": "Australia/Oceania",
    "Austria": "Europe",
    "Azerbaijan": "Asia",
    "Bahamas": "North America",
    "Bahrain": "Asia",
    "Bangladesh": "Asia",
    "Barbados": "North America",
    "Belarus": "Europe",
    "Belgium": "Europe",
    "Belize": "North America",
    "Benin": "Africa",
    "Bhutan": "Asia",
    "Bolivia": "South America",
    "Bosnia and Herzegovina": "Europe",
    "Botswana": "Africa",
    "Brazil": "South America",
    "Brunei": "Asia",
    "Bulgaria": "Europe",
    "Burkina Faso": "Africa",
    "Burma": "Asia",
    "Burundi": "Africa",
    "Cabo Verde": "Africa",
    "Cambodia": "Asia",
    "Cameroon": "Africa",
    "Canada": "North America",
    "Central African Republic": "Africa",
    "Chad": "Africa",
    "Chile": "South America",
    "China": "Asia",
    "Colombia": "South America",
    "Comoros": "Africa",
    "Congo (Brazzaville)": "Africa",
    "Congo (Kinshasa)": "Africa",
    "Costa Rica": "North America",
    "Cote d'Ivoire": "Africa",
    "Croatia": "Europe",
    "Cuba": "North America",
    "Cyprus": "Asia",
    "Czechia": "Europe",
    "Denmark": "Europe",
    "Diamond Princess": "Other",
    "Djibouti": "Africa",
    "Dominica": "North America",
    "Dominican Republic": "North America",
    "Ecuador": "South America",
    "Egypt": "Africa",
    "El Salvador": "North America",
    "Equatorial Guinea": "Africa",
    "Eritrea": "Africa",
    "Estonia": "Europe",
    "Eswatini": "Africa",
    "Ethiopia": "Africa",
    "Fiji": "Australia/Oceania",
    "Finland": "Europe",
    "France": "Europe",
    "Gabon": "Africa",
    "Gambia": "Africa",
    "Georgia": "Asia",
    "Germany": "Europe",
    "Ghana": "Africa",
    "Greece": "Europe",
    "Grenada": "North America",
    "Guatemala": "North America",
    "Guinea": "Africa",
    "Guinea-Bissau": "Africa",
    "Guyana": "South America",
    "Haiti": "North America",
    "Holy See": "Europe",
    "Honduras": "North America",
    "Hungary": "Europe",
    "Iceland": "Europe",
    "India": "Asia",
    "Indonesia": "Asia",
    "Iran": "Asia",
    "Iraq": "Asia",
    "Ireland": "Europe",
    "Israel": "Asia",
    "Italy": "Europe",
    "Jamaica": "North America",
    "Japan": "Asia",
    "Jordan": "Asia",
    "Kazakhstan": "Asia",
    "Kenya": "Africa",
    "Kiribati": "Australia/Oceania",
    "Korea, South": "Asia",
    "Kosovo": "Europe",
    "Kuwait": "Asia",
    "Kyrgyzstan": "Asia",
    "Laos": "Asia",
    "Latvia": "Europe",
    "Lebanon": "Asia",
    "Lesotho": "Africa",
    "Liberia": "Africa",
    "Libya": "Africa",
    "Liechtenstein": "Europe",
    "Lithuania": "Europe",
    "Luxembourg": "Europe",
    "MS Zaandam": "Other",
    "Madagascar": "Africa",
    "Malawi": "Africa",
    "Malaysia": "Asia",
    "Maldives": "Asia",
    "Mali": "Africa",
    "Malta": "Europe",
    "Marshall Islands": "Australia/Oceania",
    "Mauritania": "Africa",
    "Mauritius": "Africa",
    "Mexico": "North America",
    "Micronesia": "Australia/Oceania",
    "Moldova": "Europe",
    "Monaco": "Europe",
    "Mongolia": "Asia",
    "Montenegro": "Europe",
    "Morocco": "Africa",
    "Mozambique": "Africa",
    "Namibia": "Africa",
    "Nepal": "Asia",
    "Netherlands": "Europe",
    "New Zealand": "Australia/Oceania",
    "Nicaragua": "North America",
    "Niger": "Africa",
    "Nigeria": "Africa",
    "North Macedonia": "Europe",
    "Norway": "Europe",
    "Oman": "Asia",
    "Pakistan": "Asia",
    "Palau": "Australia/Oceania",
    "Panama": "North America",
    "Papua New Guinea": "Australia/Oceania",
    "Paraguay": "South America",
    "Peru": "South America",
    "Philippines": "Asia",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Qatar": "Asia",
    "Romania": "Europe",
    "Russia": "Europe",
    "Rwanda": "Africa",
    "Saint Kitts and Nevis": "North America",
    "Saint Lucia": "North America",
    "Saint Vincent and the Grenadines": "North America",
    "Samoa": "Australia/Oceania",
    "San Marino": "Europe",
    "Sao Tome and Principe": "Africa",
    "Saudi Arabia": "Asia",
    "Senegal": "Africa",
    "Serbia": "Europe",
    "Seychelles": "Africa",
    "Sierra Leone": "Africa",
    "Singapore": "Asia",
    "Slovakia": "Europe",
    "Slovenia": "Europe",
    "Solomon Islands": "Australia/Oceania",
    "Somalia": "Africa",
    "South Africa": "Africa",
    "South Sudan": "Africa",
    "Spain": "Europe",
    "Sri Lanka": "Asia",
    "Sudan": "Africa",
    "Summer Olympics 2020": "Other",
    "Suriname": "South America",
    "Sweden": "Europe",
    "Switzerland": "Europe",
    "Syria": "Asia",
    "Taiwan*": "Asia",
    "Tajikistan": "Asia",
    "Tanzania": "Africa",
    "Thailand": "Asia",
    "Timor-Leste": "Asia",
    "Togo": "Africa",
    "Tonga": "Australia/Oceania",
    "Trinidad and Tobago": "North America",
    "Tunisia": "Africa",
    "Turkey": "Asia",
    "US": "North America",
    "Uganda": "Africa",
    "Ukraine": "Europe",
    "United Arab Emirates": "Asia",
    "United Kingdom": "Europe",
    "Uruguay": "South America",
    "Uzbekistan": "Asia",
    "Vanuatu": "Australia/Oceania",
    "Venezuela": "South America",
    "Vietnam": "Asia",
    "West Bank and Gaza": "Asia",
    "Winter Olympics 2022": "Other",
    "Yemen": "Asia",
    "Zambia": "Africa",
    "Zimbabwe": "Africa"
}

df_sum = pd.read_csv(input_sum_file)

# Map countries to continents
df_sum['Continent'] = df_sum['Country'].map(countries_to_continents)

# Reorder columns to have 'Continent' as the first column
columns = ['Continent'] + [col for col in df_sum.columns if col != 'Continent']
df_sum = df_sum[columns]

# Write the updated dataframe to a new CSV file
df_sum.to_csv(output_file, index=False)

In [11]:
# Generate csv with the max total recovered and keep the associated date
output_file = "covid-data/countries-aggregated-max-recovered.csv"

# Dictionary to store the max entry data with date
max_entry_data_with_date = defaultdict(lambda: {'Total_Recovered': 0, 'Date': None})

# Read the data from the input CSV file and get the max entry for each country with date
with open(input_file, 'r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        country = row['Country']
        recovered = int(row['Recovered'])
        if recovered > max_entry_data_with_date[country]['Total_Recovered']:
            max_entry_data_with_date[country]['Total_Recovered'] = recovered
            max_entry_data_with_date[country]['Date'] = row['Date']

# Write the max entry data with date to the output CSV file
with open(output_file, 'w', newline='') as outfile:
    fieldnames = ['Date', 'Country', 'Total_Recovered']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for country, data in max_entry_data_with_date.items():
        writer.writerow({
            'Date': data['Date'],
            'Country': country,
            'Total_Recovered': data['Total_Recovered']
        })

In [12]:
from cassandra.util import uuid_from_time

input_world_file = "covid-data/worldwide-aggregate.csv"
output_file = "covid-data/worldwide-aggregate-with-timeuuid.csv"

with open(input_world_file, 'r') as csvfile, open(output_file, 'w', newline='') as outfile:
    reader = csv.DictReader(csvfile)
    fieldnames = ['timeid'] + reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in reader:
        # Generate a time-based UUID from the Date column
        date = pd.to_datetime(row['Date'])
        row['timeid'] = str(uuid_from_time(date))
        writer.writerow(row)

### Other CSVs

In [13]:
# Key Countries Pivoted
key_countries_pivoted.info()
key_countries_pivoted.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            816 non-null    datetime64[ns]
 1   China           816 non-null    int64         
 2   US              816 non-null    int64         
 3   United_Kingdom  816 non-null    int64         
 4   Italy           816 non-null    int64         
 5   France          816 non-null    int64         
 6   Germany         816 non-null    int64         
 7   Spain           816 non-null    int64         
 8   Iran            816 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 57.5 KB


,Date,China,US,United_Kingdom,Italy,France,Germany,Spain,Iran
0,2020-01-22,548,1,0,0,0,0,0,0
1,2020-01-23,643,1,0,0,0,0,0,0


In [14]:
# Reference
reference.info()
reference.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4316 entries, 0 to 4315
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   UID             4316 non-null   int64  
 1   iso2            4311 non-null   object 
 2   iso3            4312 non-null   object 
 3   code3           4312 non-null   float64
 4   FIPS            3384 non-null   float64
 5   Admin2          3343 non-null   object 
 6   Province_State  4117 non-null   object 
 7   Country_Region  4316 non-null   object 
 8   Lat             4168 non-null   float64
 9   Long_           4168 non-null   float64
 10  Combined_Key    4316 non-null   object 
 11  Population      4165 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 404.8+ KB


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0


In [15]:
# Time Series 19 Covid Combined
time_series_19_covid_combined.info()
time_series_19_covid_combined.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231744 entries, 0 to 231743
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            231744 non-null  datetime64[ns]
 1   Country/Region  231744 non-null  object        
 2   Province/State  72624 non-null   object        
 3   Confirmed       231744 non-null  int64         
 4   Recovered       218688 non-null  float64       
 5   Deaths          231744 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 10.6+ MB


,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.0,0
1,2020-01-23,Afghanistan,NaN,0,0.0,0


In [16]:
# US Confirmed
us_confirmed.info()
us_confirmed.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727072 entries, 0 to 2727071
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Admin2          object        
 1   Date            datetime64[ns]
 2   Case            int64         
 3   Country/Region  object        
 4   Province/State  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 104.0+ MB


,Admin2,Date,Case,Country/Region,Province/State
0,Autauga,2020-01-22,0,US,Alabama
1,Autauga,2020-01-23,0,US,Alabama


In [17]:
# US Deaths
us_deaths.info()
us_deaths.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727072 entries, 0 to 2727071
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Admin2          object        
 1   Date            datetime64[ns]
 2   Case            int64         
 3   Country/Region  object        
 4   Province/State  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 104.0+ MB


,Admin2,Date,Case,Country/Region,Province/State
0,Autauga,2020-01-22,0,US,Alabama
1,Autauga,2020-01-23,0,US,Alabama


In [18]:
# US Simplified
us_simplified.info()
us_simplified.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727072 entries, 0 to 2727071
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Date            datetime64[ns]
 1   Admin2          object        
 2   Province/State  object        
 3   Confirmed       int64         
 4   Deaths          int64         
 5   Country/Region  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 124.8+ MB


,Date,Admin2,Province/State,Confirmed,Deaths,Country/Region
0,2020-01-22,Autauga,Alabama,0,0,US
1,2020-01-23,Autauga,Alabama,0,0,US


In [19]:
# Worldwide Aggregate
worldwide_aggregate.info()
worldwide_aggregate.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           816 non-null    datetime64[ns]
 1   Confirmed      816 non-null    int64         
 2   Recovered      816 non-null    int64         
 3   Deaths         816 non-null    int64         
 4   Increase rate  815 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 32.0 KB


,Date,Confirmed,Recovered,Deaths,Increase rate
0,2020-01-22,557,30,17,NaN
1,2020-01-23,657,32,18,17.953321
